In [18]:
from langgraph.graph import StateGraph, END, START
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage, SystemMessage
from typing import TypedDict
import operator
from langchain_mcp_adapters.client import MultiServerMCPClient
from fastmcp import FastMCP
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
import json
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI
import os

from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS




In [19]:
load_dotenv()


True

In [20]:
llm = ChatGoogleGenerativeAI(model ="gemini-2.5-flash")

In [21]:
res = llm.invoke("who is the president of India")

In [22]:
res

AIMessage(content='The current President of India is **Droupadi Murmu**.\n\nShe assumed office on July 25, 2022, and is the 15th President of India. She is also the first tribal person and the second woman to hold the office.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019c3746-af37-7e13-8ffd-c8854c0f84a6-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 7, 'output_tokens': 205, 'total_tokens': 212, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 149}})

In [ ]:

from youtube_transcript_api import YouTubeTranscriptApi

def get_transcript(video_id: str) -> str:
    try:
        transcript_list = YouTubeTranscriptApi().fetch(video_id, languages=["en"])
        
        # Combine all text segments
        full_transcript = " ".join([segment.text for segment in transcript_list])
        
        return full_transcript
        
    except Exception as e:
        raise RuntimeError(f"could not fetch transcript: {str(e)}")


In [24]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_transcript(transcript: str):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )

    documents = text_splitter.create_documents([transcript])
    return documents

In [25]:
transcript = get_transcript("DJ3uwsESYrY")

In [26]:
transcript

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text="Hello everyone. So today I'm going to", start=0.32, duration=3.84), FetchedTranscriptSnippet(text='share my Amazon interview experience for', start=2.0, duration=4.72), FetchedTranscriptSnippet(text='ST1 role. So there were total five', start=4.16, duration=4.64), FetchedTranscriptSnippet(text='rounds. One online assessment and four', start=6.72, duration=4.64), FetchedTranscriptSnippet(text='interview rounds. I am from our tier', start=8.8, duration=5.839), FetchedTranscriptSnippet(text='three college and a 2025 graduate. I', start=11.36, duration=5.839), FetchedTranscriptSnippet(text="have applied directly through Amazon's", start=14.639, duration=5.666), FetchedTranscriptSnippet(text='career portal for SD1 role in September.', start=17.199, duration=4.641), FetchedTranscriptSnippet(text='[clears throat] And then on December', start=20.305, duration=4.894), FetchedTranscriptSnippet(text='3rd, I got an online assessment link. 

In [28]:
docs = chunk_transcript(transcript)

TypeError: expected string or bytes-like object, got 'FetchedTranscript'

In [ ]:
docs

[Document(metadata={}, page_content="Hello everyone. So today I'm going to share my Amazon interview experience for ST1 role. So there were total five rounds. One online assessment and four interview rounds. I am from our tier three college and a 2025 graduate. I have applied directly through Amazon's career portal for SD1 role in September. [clears throat] And then on December 3rd, I got an online assessment link. So the online assessment mmedly consist of two parts. The first one is coding section and the second one is work style assessment. So in the coding section there were two DSA question and both were like medium to hard level questions. So the first question was on a binary string in which I have to obtain a lexographically maximum string by repeatedly appending characters and reversing the string. [music] So this was based on greedy approach and string manipulation. The second question was like you are given a area of workers and you have to find the number of combinations of

In [ ]:
embedding_model = NVIDIAEmbeddings(model="nvidia/nv-embedqa-e5-v5")
vectorStore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute


In [ ]:
retriever = vectorStore.as_retriever(search_type = "similarity", search_kwargs={"k": 4, "lambda_mult": 0.5})

In [ ]:
result = retriever.invoke("what is provided in the video")

In [ ]:
result

[Document(id='76e47e1e-a4a1-40ce-a0f6-23f9ee7f4640', metadata={}, page_content='you are given a binary tree and you have to convert it into a [music] left skewed or right skewed binary tree in a level order traverser. So I followed BFS there and the interviewers were like pretty much uh uh impressed by the solution and then I write it down on the pen and paper. After that in the last 10 minute they asked me uh some behavioral question related to uh a leadership principle which is deliver result. So they asked me like tell me about a time when you deliver results under pressure [music] something like that. So I explained I have some stories prepared for that. So I explained those stories there. [music] After that uh waiting for like half an hour I moved forward for the second round and that was also purely DSA based. [music] Um the first question they gave me was a famous problem which is like trapping rainwater problem. So [music] I solved it using two pointer and when they satisfied w

In [ ]:
6

6